In [2]:
import sys

assert sys.version_info >= (3, 7)

In [3]:
from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [4]:
from pathlib import Path

In [5]:
# define the default font sizes to make the figures prettier:

In [6]:
tf.random.set_seed(42)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(tf.keras.layers.Dense(100,
                                    activation="swish",
                                    kernel_initializer="he_normal"))

model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [7]:
dataset = tf.keras.datasets.cifar10.load_data()

In [8]:
optimizer = tf.keras.optimizers.legacy.Nadam(learning_rate=5e-5) #Legacy cause made on a m2
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [9]:
(X_train_full, y_train_full), (X_test, y_test) = dataset

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

In [10]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_cifar10_model.h5",
                                                         save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = Path() / "my_cifar10_logs" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [11]:
%load_ext tensorboard
%tensorboard --logdir=./my_cifar10_logs

In [12]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 8s 5ms/step - loss: 4.4978 - accuracy: 0.1753 - val_loss: 2.2195 - val_accuracy: 0.2096
Epoch 2/100
  34/1407 [..............................] - ETA: 6s - loss: 2.1371 - accuracy: 0.2243

/opt/anaconda3/envs/homl3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1407/1407 [==============================] - 7s 5ms/step - loss: 2.0792 - accuracy: 0.2329 - val_loss: 2.1202 - val_accuracy: 0.2254
Epoch 3/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.9778 - accuracy: 0.2678 - val_loss: 1.9736 - val_accuracy: 0.2698
Epoch 4/100
1407/1407 [==============================] - 7s 5ms/step - loss: 1.9024 - accuracy: 0.3040 - val_loss: 1.9797 - val_accuracy: 0.2988
Epoch 5/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.8394 - accuracy: 0.3285 - val_loss: 1.8609 - val_accuracy: 0.3216
Epoch 6/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.7923 - accuracy: 0.3498 - val_loss: 1.8225 - val_accuracy: 0.3358
Epoch 7/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.7467 - accuracy: 0.3690 - val_loss: 1.7588 - val_accuracy: 0.3588
Epoch 8/100
1407/1407 [==============================] - 8s 5ms/step - loss: 1.7087 - accuracy: 0.3856 - val_loss: 1.6960 - val_accuracy: 0.39

In [13]:
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 1.5314 - accuracy: 0.4710


[1.531402349472046, 0.47099998593330383]